In [ ]:
!nvidia-smi

In [ ]:
from IPython.display import clear_output
import os
import yaml

from ultralytics import YOLO
# import torch
# import torch.nn as nn

In [ ]:
import ultralytics

ultralytics.checks()

In [ ]:
!pwd

In [ ]:
!cd /project/lt900101-ai24e1/

In [ ]:
# Define your data dictionary
data = {
  'train': '/project/lt900101-ai24e1/liver-ultrasound-detection/train/train',  # Path to training data
  'val': '/project/lt900101-ai24e1/liver-ultrasound-detection/val/val',    # Path to validation data
  'nc': 7,                        # Number of classes
  'names': [0, 1, 2, 3, 4, 5, 6]  # Class names
}

# Set the filename
filename = 'dataset.yaml'

# Write the data dictionary to a YAML file
with open(filename, 'w') as f:
  yaml.dump(data, f)

print(f'YAML file created: {filename}')

In [ ]:
# Load a model
# model = YOLO("yolov8n.yaml")  # build a new model from YAML
# model = YOLO("yolov8n.pt")  # load a pretrained model (recommended for training)
# model = YOLO("/content/-q/yolov10x.pt").load("yolov8x.pt")  # build from YAML and transfer weights

# model = YOLO("/project/lt900101-ai24e1/Atom/-q/yolov10x.pt").load("yolov8x.pt")  # build from YAML and transfer weights

# model = YOLO("/project/lt900101-ai24e1/Atom/-q/yolov10x.pt")  # build from YAML and transfer weights

In [ ]:
#from ultralytics import YOLO
#just try to change annotations -> labels (in all train and val always delete .cache file in train and val)

model = YOLO('yolov8x.pt')
params = {
    "task": "detect",
    "mode": "train",
    "model": "yolov8x.pt",
    "data": "dataset.yaml",
    "epochs": 100,
    "batch": 64,
    "agnostic_nms": True,
    "cos_lr": True,
    #"optimizer" : 'AdamW', #require high-ram can be max 36.9
    "dropout" : 0.1,
    "conf" : 0.54,
    "device" : [0,1,2,3],
    "augment": False  # Disable augmentation
    #add no augmentation

}

model.train(**params)

### Modify submission expanding bbox

In [ ]:
import pandas as pd
import ast

In [ ]:
submission = pd.read_csv("/content/RT-DETR-L-Cof05.csv")
x = ast.literal_eval(submission["Annotation"][0])
type(x[0][0])

In [ ]:
submission

In [ ]:
for i in range(len(submission)):
    labels = submission["Label"][i]
    if(labels!='[]'):
        ls = ast.literal_eval(submission["Annotation"][i])
        ls_new = []
        for idx in range(len(ls)):
            sub_ls = ls[idx]
            sub_ls[2] *= 1.5
            sub_ls[3] *= 1.5
            ls_new.append(sub_ls)
        submission["Annotation"][i] = ls_new

In [ ]:
submission.to_csv("modify-1.csv", index=False)

### Make submission

#### Import

In [ ]:
#!pip install -q git+https://github.com/THU-MIG/yolov10.git
#!pip install ultralytics
#!pip install gdown

In [ ]:
import torch
import ultralytics

import pandas as pd
import cv2
import json
import numpy as np

import os

from ultralytics import RTDETR
from ultralytics import YOLOv10

In [ ]:
print(f'Setup complete. Using torch {torch.__version__}')
print(f"{torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'}")

In [ ]:
ultralytics.checks()

In [ ]:
df = pd.read_excel('/project/lt900101-ai24e1/liver-ultrasound-detection/mapping.xlsx')
df

In [ ]:
!pwd

In [ ]:
#!gdown 1-9VhoLpTYHhR_5-NQAdop-yM7PZH0E6x

In [ ]:
#!unzip /content/smooth_clahe_test.zip

In [ ]:
images_dir = '/project/lt900101-ai24e1/liver-ultrasound-detection/test/test/images'
# images_dir = '/project/lt900101-ai24e1/super_contrast_test/images'
images_files = sorted([f for f in os.listdir(images_dir) if f.endswith('.jpg')])
len(images_files)

In [ ]:
weights_path = '/project/lt900101-ai24e1/Atom/runs/detect/train/weights/best.pt'
model = YOLOv10(weights_path)

def detect_objects(image_path, model):
    results = model(source=image_path, conf=0.5274)
    boxes = results[0].boxes.xyxy[:, :4].cpu().numpy()
    classes = results[0].boxes.cls.cpu().numpy()
    scores = results[0].boxes.conf.cpu().numpy()
    boxes_n = []
    for boxe in boxes:
      boxe = [float(i) for i in boxe]
      boxes_n.append(boxe)
    classes = [int(classe) for classe in classes]
    scores = [round(score, 4) for score in scores]
    return boxes_n, classes, scores

df = pd.read_csv('/project/lt900101-ai24e1/liver-ultrasound-detection/sample_submission.csv')
df

In [ ]:
help(model)

In [ ]:
model.predict(source='/project/lt900101-ai24e1/liver-ultrasound-detection/test/test/images/18999.jpg', conf=0.5)

In [ ]:
import glob
all_path = glob.glob("/content/images/*")
print('/content/images/18999.jpg' in all_path)

In [ ]:
from tqdm.auto import tqdm

b = []
c= []
counter = 0

for path in tqdm(df['Image File']):
    image_path = '/project/lt900101-ai24e1/liver-ultrasound-detection/test/test/images/{0}.jpg'.format(path)
    #image_path = '/project/lt900101-ai24e1/super_contrast_test/images/{0}.jpg'.format(path)
    print(image_path)
    boxes, classes, scores = detect_objects(image_path, model)
    b.append(boxes)
    c.append(classes)


df['Annotation'] = b
df['Label'] = c
df

In [ ]:
import numpy as np

In [ ]:
uni = []

for j in range(len(df)):
    if(df["Annotation"][j] not in uni):
        uni.append(df["Annotation"][j])

In [ ]:
print(uni)
print(len(uni))

In [ ]:
df.to_csv("submission_yolov8x_100_epoch_raw.csv", index=False)